In [19]:
import numpy as np
import time
import random

class ModeleFatigueAmortisseur:
    def __init__(self, Sut=600, d=20, surface='usinee', R=0.95, T=25, Esalt=0.2, H=60, Kt=1.5, q=0.75):
        self.Sut = Sut
        self.d = d
        self.surface = surface
        self.R = R
        self.T = T
        self.Esalt = Esalt
        self.H = H
        self.Kt = Kt
        self.q = q
        self.D_cumule = 0.0 
        self.rdamage = 1e-6 
        self.m = 5           

    # --- Facteurs correctifs ---
    def facteur_surface(self):
        coeff = {'poli': (1.58, -0.085), 'usinee': (4.51, -0.265), 'laminee': (57.7, -0.718)}
        a, b = coeff.get(self.surface, (4.51, -0.265))
        return a * (self.Sut ** b)

    def facteur_taille(self):
        if self.d <= 8:
            return 1.0
        elif self.d <= 250:
            return 1.189 * (self.d ** -0.097)
        return 0.6

    def facteur_fiabilite(self):
        z_table = {0.999: 0.753, 0.99: 0.814, 0.95: 0.868, 0.90: 0.897}
        return z_table.get(round(self.R, 3), 0.868)

    def facteur_cc_dyn(self):
        return 1/(self.q*(self.Kt-1)+1)
    def facteur_temperature(self):
        if self.T <= 71:
            return 1.0
        elif self.T > 71:
            return 344/(self.T+273)
        return 1.0

    def facteur_corrosion(self):
        return 1/(1 + 0.3 * self.Esalt + 0.1 * self.H / 100)
    @staticmethod
    def limite_fatigue_corrigee(self):
        sigmaD_prime=0.5 * self.Sut
        Ks = self.facteur_surface()
        Kg = self.facteur_taille()
        KF = self.facteur_fiabilite()
        Kc = self.facteur_cc_dyn()
        KT = self.facteur_temperature()
        Kd = self.facteur_corrosion()
        return sigmaD_prime * Ks * Kg * KF * KT * Kd * Kc
    

    # Calculs contrainte-durée de vie
    def contrainte_tige(self, F):
        A = np.pi * (self.d / 2) ** 2 
        return (F / A) * self.Kt 

    def cycles_rupture(self, sigma):
        sigmaD = self.limite_fatigue_corrigee(self)
        N = (10**3) * ((0.9 * self.Sut) / sigma) ** (3.0 / np.log10((0.9 * self.Sut) / sigmaD))
        return N

    def mise_a_jour_dommage(self, sigma):
        Nf = self.cycles_rupture(self, sigma)
        dommage_onecycle = 1 / Nf  
        self.D_cumule += dommage_onecycle
        return self.D_cumule

    def duree_vie_restante(self):
        Dfailure = 1 - (1 - self.R) ** 2 / 2
        return max((Dfailure - self.D_cumule) / self.rdamage, 0)

    def proba_defaillance(self):
        alpha, D50 = 10, 0.8
        return 1 / (1 + np.exp(-alpha * (self.D_cumule - D50)))




In [4]:
#essai
sigma = 600
Sut = 1000
sigmaD = 500
# use ** for power (not ^ which is bitwise xor), ensure float arithmetic
N = (10**3) * ((0.9 * Sut) / sigma) ** (3.0 / np.log10((0.9 * Sut) / sigmaD))
print(N)

117341.15490905166


In [20]:
# --- Simulation temps réel ---
def simulation_temps_reel(n_points=1000, delai=0.1):
    modele = ModeleFatigueAmortisseur(d=18, R=0.95, surface='usinee', T=35, Esalt=0.4, H=75)
    print("=== DÉBUT DE LA SURVEILLANCE TEMPS RÉEL ===\n")

    for i in range(n_points):
        F = random.uniform(500, 2500)  # N (Normalement on aura besoin des données d'un vrai capteur ici, c'est juste une simulation)
        sigma = modele.contrainte_tige(F)
        D = modele.mise_a_jour_dommage(sigma)
        vie = modele.duree_vie_restante()
        Pfail = modele.proba_defaillance()

        print(f"Échantillon {i+1:03d} | F={F:7.1f} N | σ={sigma:6.2f} MPa | D={D:.6f} | Vie restante≈{vie:.2f} | Pfail={Pfail*100:.2f}%")

        # Condition d’alerte
        if D >= 0.9:
            print("\n⚠️  ALERTE : Seuil de défaillance atteint ! Remplacer l’amortisseur.")
            break

        time.sleep(delai) 

    print("\n=== FIN DE LA SIMULATION ===")


# --- Exécution ---
if __name__ == "__main__":
    simulation_temps_reel(n_points=200, delai=0.05)

=== DÉBUT DE LA SURVEILLANCE TEMPS RÉEL ===



TypeError: ModeleFatigueAmortisseur.cycles_rupture() takes 2 positional arguments but 3 were given

In [3]:
# ================== Exemple d'utilisation =====================
modele = ModeleFatigueAmortisseur(Sut=700, d=18, surface='usinee', R=0.95, T=30, Esalt=0.3, H=70)

# Données d’essai :
forces = [1000, 1500, 2000, 2500]  # N
contraintes = [modele.contrainte_tige(F, Kt=1.5) for F in forces]
cycles = [1e4, 5e4, 1e5, 2e5]

D = modele.dommage_miner(contraintes, cycles)
vie_restante = modele.duree_vie_restante(D_current=D, rdamage=1e-6)
P_fail = modele.proba_defaillance(D)

print(f"Contrainte moyenne : {np.mean(contraintes):.2f} MPa")
print(f"Dommage cumulé : {D:.4f}")
print(f"Durée de vie restante estimée : {vie_restante:.2f} unités")
print(f"Probabilité de défaillance : {P_fail*100:.2f} %")


Contrainte moyenne : 10.32 MPa
Dommage cumulé : 0.1645
Durée de vie restante estimée : 834204.02 unités
Probabilité de défaillance : 0.17 %
